# Part 1: Collecting Data from Twitter

<img src="https://upload.wikimedia.org/wikipedia/commons/6/63/Twitter-logo-png-5859_-_Copy.png">

**Suggested Readings:** 
* Chapter 1 and Chapter 9 of the book "Mining the Social Web" (https://miningthesocialweb.com/) 
can help a lot if you get stuck. 
* In fact, it is intentional that many of these questions can be answered directly from there (except for question 4)!
* The idea is to ease you into workshop :-)

**Don't forget!**
* You will need to install the "twitter" library to access the Twitter API
 * pip install twitter

** NOTE **
* **Please don't forget to save the notebook frequently when working in IPython Notebook, otherwise the changes you made can be lost**.
* **There is a package called python-twitter which is wonderful, but has a very different API.  If you get the wrong one then my examples will not work!**

# Sampling Twitter Data with Either Streaming or Search API about a certain topic

* Select a topic that you are interested in, for example, "WPI" or "Lady Gaga"
* Use Twitter Streaming API to sample a collection of tweets about this topic. (It would be recommended that the number of tweets should be larger than 1,000, but smaller than 10,000).
* Store the tweets you downloaded into a local file (txt file or json file) 

In [20]:
import twitter
import os
#---------------------------------------------
# Define a Function to Login Twitter API
def oauth_login():
    # Prof. Paffenroth has a developer account for the class.  He will provide the Twitter access tokens for
    # each team
    # See https://developer.twitter.com/docs/auth/oauth for more information 
    # on Twitter's OAuth implementation.
    
    # Team 1-3
    CONSUMER_KEY = 'MbhA6iSZflOyYErveZzuNosE4'
    CONSUMER_SECRET ='zvz5i6WBvS81EGkYQKMArXmnvL1vaVXXAZfskFclAeUgrU2twR'
    OAUTH_TOKEN = '571213367-umrVxIxuWiE1uzGOw279gdsvecNQBXdab7CfqZuy'
    OAUTH_TOKEN_SECRET = 'BI7pACPjUhZJtNNFJOZ1PLV2lflZzzEA16lLp7oFWsDvU'

#     # Team 4-6
#     CONSUMER_KEY = 'YirQT0fCiZdNIJRDWXFpwU9OU'
#     CONSUMER_SECRET ='tJByz9a9JNwWCu3iH0P0CXa1t0AugfgFRiGifKZfD8Lgm8glgS'
#     OAUTH_TOKEN = '571213367-lxmjiUqnyn7hmt9CL2CBMq5kRrDhMjAS7xLcaVcx'
#     OAUTH_TOKEN_SECRET = 'FoLCxcmJWuOKLAIyXuxwtG3xpQBIM2qsUteXMCF7FxJGG'

#     # Team 7-9
#     CONSUMER_KEY = 'dyqx8OLPb7PeGkOCUOPZPHsGQ'
#     CONSUMER_SECRET ='LM4SBcEbEHTLdJXyFRVOyBGl0TAjXmS48djm3XweYv3pBxjhwL'
#     OAUTH_TOKEN = '571213367-W8ejT9u0M77lN52v82UeIIrgujO6smyrCNlauRee'
#     OAUTH_TOKEN_SECRET = '7l1mvstsjPBenq6Sa9fuu8X0M3UVVgo3CF7fQBM0uPUVW'

#     # Team 10-12
#     CONSUMER_KEY = 'ijh1zy12m73JOIooMbx56tB8B'
#     CONSUMER_SECRET ='j3fyLkFMOtcRUV3swUlzvOZayiAbSBvMXzeC2bDfdayUD0mhpM'
#     OAUTH_TOKEN = '571213367-szzTWKXISKZeytQRl0z4P1giPdAh3OqTpfQRhj0g'
#     OAUTH_TOKEN_SECRET = 'zx0VyALB2OO2y0vmHPcz8W612WSSnoTgGIyZcQD0w6Mh2'

#     # Team 13-15
#     CONSUMER_KEY = '9ooQGUNzPRUjdnZTZeI7wrL4Z'
#     CONSUMER_SECRET ='jURzPZrJ33yYxyQSKXOHw66n71jdCC3ge9omXlENcZXvR3zzaq'
#     OAUTH_TOKEN = '571213367-uIPN0Yck4PBhakA2ZksskGm37qWZ8IOMCQF5pUpa'
#     OAUTH_TOKEN_SECRET = 'pJhzPUKJLHv0sFqHzh1iGq97p9BC9ACuIsqAbTtiiUtxz'

    # Emergency - break glass to use!
    #CONSUMER_KEY = 'KRxduP2c4L9mFv5OwezcATMd4'
    #CONSUMER_SECRET ='Sjpg0F64ElNGMainYvclqB5sRHKfmXwUWuRDyPL26erwCF1jQ1'
    #OAUTH_TOKEN = '571213367-ajwp6Q8xW5BkUvamespO9ezC6lK6ZjXRguwevMkI'
    #OAUTH_TOKEN_SECRET = 'V9ydSDjQf4ELIXT0fEuSKs3blBYBj7tcFyEKGRjh383JG'

    
    auth = twitter.oauth.OAuth(OAUTH_TOKEN, OAUTH_TOKEN_SECRET,
                               CONSUMER_KEY, CONSUMER_SECRET)
    
    twitter_api = twitter.Twitter(auth=auth)

    return twitter_api

twitter_api = oauth_login()

#----------------------------------------------
# Your code starts here
#   Please add comments or text cells in between to explain the general idea of each block of the code.
#   Please feel free to add more cells below this cell if necessary
import json
def twitter_search(twitter_api, q, max_results=200, **kw):

    # See http://bit.ly/2QyGz0P and https://bit.ly/2QyGz0P
    # for details on advanced search criteria that may be useful for
    # keyword arguments

    # See https://dev.twitter.com/docs/api/1.1/get/search/tweets
    search_results = twitter_api.search.tweets(q=q, count=100, **kw)

    statuses = search_results['statuses']

    # Iterate through batches of results by following the cursor until we
    # reach the desired number of results, keeping in mind that OAuth users
    # can "only" make 180 search queries per 15-minute interval. See
    # https://developer.twitter.com/en/docs/basics/rate-limits
    # for details. A reasonable number of results is ~1000, although
    # that number of results may not exist for all queries.

    # Enforce a reasonable limit
    max_results = min(1000, max_results)

    for _ in range(10): # 10*100 = 1000
        try:
            next_results = search_results['search_metadata']['next_results']
        except KeyError as e: # No more results when next_results doesn't exist
            break

        # Create a dictionary from next_results, which has the following form:
        # ?max_id=313519052523986943&q=NCAA&include_entities=1
        kwargs = dict([ kv.split('=')
                        for kv in next_results[1:].split("&") ])

        search_results = twitter_api.search.tweets(**kwargs)
        statuses += search_results['statuses']

        if len(statuses) > max_results:
            break

    return statuses

In [4]:
# Sample usage

twitter_api = oauth_login()

# You can change this to be any string you like!
q = "Samsung"
results = twitter_search(twitter_api, q, max_results=1000)
print(results[0])

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



You will thank me for this, and your students will in turn thank you :-).  This is how you can key a running list of the tweets you gather and keep a library of tweets around when things fail!  Note, this assumes that results has new data in it.  Otherwise you will just get multiple copies of the same tweets.

In [54]:
def appendTweets(filename):
    if os.path.exists(filename):
        with open(filename, 'r+') as file:
            # First we load existing data into a dict.
            file_data = json.load(file)
            # Join new_dat3a with file_data
            file_data += results
            # Sets file's current position at offset.
            file.seek(0)
            # convert back to json.
            json.dump(file_data, file, indent = 1)
    else:
        with open(filename, 'w+') as file:
            # convert back to json.
            json.dump(results, file, indent = 1)

A file in which to store the downloaded tweets

In [56]:
filename = 'my_precious_tweets_on_%s.json'%'test2'
appendTweets(filename)

## This is what I do (and have done for you :-) for getting an emergency collection of tweets.

Gathering real data very often will run into some type of issue.  For example, lossing network connectivity.  Gathering tweets is even more tricky, since Twitter enforces rate limits.  I.e., if you gather too many tweets then Twitter will unceremoniously cut you off.  Accordingly, it is good to keep all the tweets you gather for later processing.  Here is a little loop that will gather a group of tweets on various topics.

In [57]:
type(results)

list

In [14]:
# # Commented out so you don't run by mistake.  This will very quickly get you rate limited.
# searches= ['Toyota', 'LG', 'election', 'restaurant', 'pizza', 'MtG', 'WoW', 'Ford', 'chocolate', 'pokemon']

# for q in searches:
#     results = twitter_search(twitter_api, q, max_results=1000)
#     filename = 'my_precious_tweets_on_%s.json'%q
#     appendTweets(filename)

# Basic Analysis of Tweets and Tweet Entities with Frequency Analysis

First, read back in your tweets

In [10]:
filename = 'my_precious_tweets_on_%s.json'%q
results = json.load(open(filename,'r'))

## Find the most popular tweets in your collection of tweets

Please plot a table of the top 10 tweets that are the most popular among your collection, i.e., the tweets with the largest number of retweet counts.


In [ ]:
#----------------------------------------------
# Your code starts here
#   Please add comments or text cells in between to explain the general idea of each block of the code.
#   Please feel free to add more cells below this cell if necessary
def find_popular_tweets(twitter_api, statuses, retweet_threshold=3):

    # You could also consider using the favorite_count parameter as part of
    # this heuristic, possibly using it to provide an additional boost to
    # popular tweets in a ranked formulation

    return [ status
                for status in statuses
                    if status['retweet_count'] > retweet_threshold ]

# Sample usage

popular_tweets = find_popular_tweets(twitter_api, results)

for tweet in popular_tweets:
    print(tweet['text'], tweet['retweet_count'])



## Find the most popular Tweet Entities in your collection of tweets

Please plot a table of the top 10 hashtags, top 10 user mentions that are the most popular in your collection of tweets.

In [ ]:
#----------------------------------------------
# Your code starts here
#   Please add comments or text cells in between to explain the general idea of each block of the code.
#   Please feel free to add more cells below this cell if necessary

from collections import Counter

def extract_tweet_entities(statuses):

    # See https://bit.ly/2MELMkm
    # for more details on tweet entities

    if len(statuses) == 0:
        return [], [], [], [], []

    screen_names = [ user_mention['screen_name']
                         for status in statuses
                            for user_mention in status['entities']['user_mentions'] ]

    hashtags = [ hashtag['text']
                     for status in statuses
                        for hashtag in status['entities']['hashtags'] ]

    urls = [ url['expanded_url']
                     for status in statuses
                        for url in status['entities']['urls'] ]

    # In some circumstances (such as search results), the media entity
    # may not appear
    medias = []
    symbols = []
    for status in statuses:
        if 'media' in status['entities']:
            for media in status['entities']['media']:
                medias.append(media['url'])
        if 'symbol' in status['entities']:
            for symbol in status['entities']['symbol']:
                symbols.append(symbol)

    return screen_names, hashtags, urls, medias, symbols

# Sample usage
screen_names, hashtags, urls, media, symbols = extract_tweet_entities(results)

# Explore the first five items for each...

print(json.dumps(screen_names[0:5], indent=1))
print(json.dumps(hashtags[0:5], indent=1))
print(json.dumps(urls[0:5], indent=1))
print(json.dumps(media[0:5], indent=1))
print(json.dumps(symbols[0:5], indent=1))

def get_common_tweet_entities(statuses, entity_threshold=3):

    # Create a flat list of all tweet entities
    tweet_entities = [  e
                        for status in statuses
                            for entity_type in extract_tweet_entities([status])
                                for e in entity_type
                     ]

    c = Counter(tweet_entities).most_common()

    # Compute frequencies
    return [ (k,v)
             for (k,v) in c
                 if v >= entity_threshold
           ]

# Sample usage
common_entities = get_common_tweet_entities(results)

print("Most common tweet entities")
print(common_entities)



# Getting "All" friends and "All" followers of a popular user in twitter


* choose a popular twitter user who has many followers, such as "ladygaga".
* Get the list of all friends and all followers of the twitter user.
* Plot 20 out of the followers, plot their ID numbers and screen names in a table.
* Plot 20 out of the friends (if the user has more than 20 friends), plot their ID numbers and screen names in a table.

In [ ]:
import sys
import time
from urllib.error import URLError
from http.client import BadStatusLine
import json
import twitter

def make_twitter_request(twitter_api_func, max_errors=10, *args, **kw):

    # A nested helper function that handles common HTTPErrors. Returns an updated
    # value for wait_period if the problem is a 500-level error. Blocks until the
    # rate limit is reset if it's a rate-limiting issue (429 error). Returns None
    # for 401 and 404 errors, which require special handling by the caller.
    def handle_twitter_http_error(e, wait_period=2, sleep_when_rate_limited=True):

        if wait_period > 3600: # Seconds
            print('Too many retries. Quitting.', file=sys.stderr)
            raise e

        # See https://developer.twitter.com/en/docs/basics/response-codes
        # for common codes

        if e.e.code == 401:
            print('Encountered 401 Error (Not Authorized)', file=sys.stderr)
            return None
        elif e.e.code == 404:
            print('Encountered 404 Error (Not Found)', file=sys.stderr)
            return None
        elif e.e.code == 429:
            print('Encountered 429 Error (Rate Limit Exceeded)', file=sys.stderr)
            if sleep_when_rate_limited:
                print("Retrying in 15 minutes...ZzZ...", file=sys.stderr)
                sys.stderr.flush()
                time.sleep(60*15 + 5)
                print('...ZzZ...Awake now and trying again.', file=sys.stderr)
                return 2
            else:
                raise e # Caller must handle the rate-limiting issue
        elif e.e.code in (500, 502, 503, 504):
            print('Encountered {0} Error. Retrying in {1} seconds'\
                  .format(e.e.code, wait_period), file=sys.stderr)
            time.sleep(wait_period)
            wait_period *= 1.5
            return wait_period
        else:
            raise e

    # End of nested helper function

    wait_period = 2
    error_count = 0

    while True:
        try:
            return twitter_api_func(*args, **kw)
        except twitter.api.TwitterHTTPError as e:
            error_count = 0
            wait_period = handle_twitter_http_error(e, wait_period)
            if wait_period is None:
                return
        except URLError as e:
            error_count += 1
            time.sleep(wait_period)
            wait_period *= 1.5
            print("URLError encountered. Continuing.", file=sys.stderr)
            if error_count > max_errors:
                print("Too many consecutive errors...bailing out.", file=sys.stderr)
                raise
        except BadStatusLine as e:
            error_count += 1
            time.sleep(wait_period)
            wait_period *= 1.5
            print("BadStatusLine encountered. Continuing.", file=sys.stderr)
            if error_count > max_errors:
                print("Too many consecutive errors...bailing out.", file=sys.stderr)
                raise

# Sample usage

twitter_api = oauth_login()

# See http://bit.ly/2Gcjfzr for twitter_api.users.lookup

response = make_twitter_request(twitter_api.users.lookup,
                                screen_name="ladygaga")

print(json.dumps(response, indent=1))

In [48]:
from functools import partial
from sys import maxsize as maxint

def get_friends_followers_ids(twitter_api, screen_name=None, user_id=None,
                              friends_limit=maxint, followers_limit=maxint):

    # Must have either screen_name or user_id (logical xor)
    assert (screen_name != None) != (user_id != None), "Must have screen_name or user_id, but not both"

    # See http://bit.ly/2GcjKJP and http://bit.ly/2rFz90N for details
    # on API parameters

    get_friends_ids = partial(make_twitter_request, twitter_api.friends.ids,
                              count=5000)
    get_followers_ids = partial(make_twitter_request, twitter_api.followers.ids,
                                count=5000)

    friends_ids, followers_ids = [], []

    for twitter_api_func, limit, ids, label in [
                    [get_friends_ids, friends_limit, friends_ids, "friends"],
                    [get_followers_ids, followers_limit, followers_ids, "followers"]
                ]:

        if limit == 0: continue

        cursor = -1
        while cursor != 0:

            # Use make_twitter_request via the partially bound callable
            if screen_name:
                response = twitter_api_func(screen_name=screen_name, cursor=cursor)
            else: # user_id
                response = twitter_api_func(user_id=user_id, cursor=cursor)

            if response is not None:
                ids += response['ids']
                cursor = response['next_cursor']

            print('Fetched {0} total {1} ids for {2}'.format(len(ids),
                  label, (user_id or screen_name)),file=sys.stderr)

            # You may want to store data during each iteration to provide an
            # additional layer of protection from exceptional circumstances

            if len(ids) >= limit or response is None:
                break

    # Do something useful with the IDs, like store them to disk
    return friends_ids[:friends_limit], followers_ids[:followers_limit]

# Sample usage

twitter_api = oauth_login()

friends_ids, followers_ids = get_friends_followers_ids(twitter_api,
                                                       screen_name="randypaffenroth",
                                                       # screen_name="ladygaga",
                                                       friends_limit=100,
                                                       followers_limit=100)

print(friends_ids)
print(followers_ids)



Fetched 21 total friends ids for randypaffenroth


[16687357, 3348826288, 40414138, 333547512, 323905099, 14755031, 119437491, 47042324, 326164900, 139790231, 466959826, 450725339, 7144422, 783214, 20536157, 14216123, 20402945, 141403992, 50393960, 14009672, 74371908]
[2458724984, 3302333726, 4460376852, 1307064024199311367, 1198093452585656320, 423676582, 31067209, 3011086935, 33891267, 1260657884, 450725339, 2533071763, 1078413485690769409, 1554296732, 65777465, 839933639450312706, 996818600517238784, 3171620876, 3464328314, 750708195539099648, 824473421476143105, 746046557548584965, 94022997, 1183815506, 159883398, 1529017262, 189581991, 14319272, 1241871145, 466959826, 47042324, 14009672]


Fetched 32 total followers ids for randypaffenroth


* Compute the mutual friends within the two groups, i.e., the users who are in both friend list and follower list, plot their ID numbers and screen names in a table

In [21]:
# Sample usage

twitter_api = oauth_login()

friends_ids_1, followers_ids_1 = get_friends_followers_ids(twitter_api,
                                                       screen_name="ladygaga",
                                                       friends_limit=100,
                                                       followers_limit=100)
friends_ids_2, followers_ids_2 = get_friends_followers_ids(twitter_api,
                                                       screen_name="randypaffenroth",
                                                       friends_limit=100,
                                                       followers_limit=100)

Fetched 5000 total friends ids for ladygaga
Fetched 5000 total followers ids for ladygaga
Fetched 21 total friends ids for randypaffenroth
Fetched 31 total followers ids for randypaffenroth


In [22]:
friends_ids_1 = set(friends_ids_1)
followers_ids_1 = set(followers_ids_1)
friends_ids_2 = set(friends_ids_2)
followers_ids_2 = set(followers_ids_2)

print(friends_ids_1.intersection(friends_ids_2))
print(followers_ids_1.intersection(followers_ids_2))


set()
set()


# Business question! 

Run some additional experiments with your data to gain familiarity with the twitter data and twitter API.

* Come up with a business question that Twitter data could help answer.
* Decribe the business case.
* How could Twitter data help a company decide how to spend its resources.

In [50]:
len(response)

1

In [51]:
import pandas as pd
tweet_results = pd.DataFrame(response)

In [52]:
tweet_results.columns

Index(['id', 'id_str', 'name', 'screen_name', 'location', 'description', 'url',
       'entities', 'protected', 'followers_count', 'friends_count',
       'listed_count', 'created_at', 'favourites_count', 'utc_offset',
       'time_zone', 'geo_enabled', 'verified', 'statuses_count', 'lang',
       'status', 'contributors_enabled', 'is_translator',
       'is_translation_enabled', 'profile_background_color',
       'profile_background_image_url', 'profile_background_image_url_https',
       'profile_background_tile', 'profile_image_url',
       'profile_image_url_https', 'profile_link_color',
       'profile_sidebar_border_color', 'profile_sidebar_fill_color',
       'profile_text_color', 'profile_use_background_image',
       'has_extended_profile', 'default_profile', 'default_profile_image',
       'following', 'follow_request_sent', 'notifications', 'translator_type',
       'withheld_in_countries'],
      dtype='object')

In [46]:
tweet_results.head()

,id,id_str,name,screen_name,location,description,url,entities,protected,followers_count,...,profile_text_color,profile_use_background_image,has_extended_profile,default_profile,default_profile_image,following,follow_request_sent,notifications,translator_type,withheld_in_countries
0,14230524,14230524,Lady Gaga,ladygaga,,“Chromatica” ⚔️💓 OUT NOW https://t.co/dgVb2wLj...,https://t.co/r5yRJxbDmD,{'url': {'urls': [{'url': 'https://t.co/r5yRJx...,False,83622167,...,333333,True,False,False,False,False,False,False,regular,[]
